In [ ]:
import pandas as pd
import numpy as np 
#from faker import Faker
import re, json, logging , html, os
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_parser import issue_parser
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.dbupdate_parser import dbupdate_parser
from lib.script_generator import script_generator 
from lib.db_parser import db_parser   
from bs4 import BeautifulSoup    
#from nltk.corpus import stopwords 
#sw=stopwords.words('english') 
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
txt = '8433'
with open ('cache/8433.html','r') as f:
    txt=f.read()
    txt=re.sub('&nbsp;', ' ', txt) 
    txt=re.sub('<p>\s*</p>', ' ', txt)  
    txt=re.sub('\n\s*\n', '\n', txt)
    txt=re.sub('\n{1,}', '\n', txt)
    
print(txt)
lines = re.split('(?:(\n?.*>?\d[\d\.a-z]+))\s*', txt )
metrics = zip(lines[1::2], lines[2::2])
metrics = [f'{x} {y}' for x,y  in list(metrics)]
metrics



In [ ]:
issue_parser(ctx).parse('8787:8790')

# PIPELINE

#### requirements > jira > sql > forms
#### requirements/jira > sql > forms

# forms from sql

In [ ]:
parser = db_parser(ctx)
df=parser.parse(question_group='4406')
df

In [ ]:
code=script_generator(ctx).generate(df) 
print(code)

### Jira Parse Entire Datacall

In [ ]:
ip = issue_parser(ctx)     
df=ip.parse('9600:9605' )  
df

### sql from jira

In [ ]:
gen=script_generator(ctx)
code=gen.generate(df, code_template_path=f'{ctx.get_tempalte_dir()}aspx\\ig.sql')
print(code)

### NLP

In [ ]:
string = 'Effective / Not Effective'
picklist = string.split(' / ')
print ( picklist ) 
pr = picklist_recommender(ctx, use_cache=False) 
pl=pr.recommend(picklist, generate_sql=False)
pl

In [ ]:
class aspx_generator():
    def __init__(self, ctx):  
        self.ctx=ctx 
    def generate(self, PK_FORM='2023-A-HVA', classname=None, where=None, commit=False, PageNameFilter='~', dest=None ):
        parser=db_parser(ctx)
        generator=script_generator(ctx)
        sql= f"""
            SELECT DISTINCT PK_FORM, PK_QuestionGroup, SectionNum, GroupName, ASPX
            FROM vwQuestions  WHERE 1=1 AND PK_FORM='{PK_FORM}' 
            --AND {where}
            ORDER BY PK_QuestionGroup ASC
        """
        if classname == None: classname=PK_FORM.replace('-','_')
        if where != None: sql=sql.replace('--AND', 'AND  ')
        #print(sql)
        df=sql_todf(sql, self.ctx.connstr)     
        df['PNAME']=df['ASPX'].apply(lambda s: s.split('/')[len(s.split('/'))-1].replace('.aspx',''))
        pages=df.to_dict(orient='records')    
        src=ctx.get_tempalte_dir()
        out=''
        for p in pages:   
            PageName = re.sub(PageNameFilter,'',f"{classname}_{p['GroupName']}")
            for ext in ['.aspx','.aspx.designer.vb','.aspx.vb']:
                with open(f'{src}{classname}{ext}', 'r', encoding='UTF-8', errors='ignore') as fr: 
                    txt = fr.read().encode(  'utf-8', errors='ignore' ).decode('utf-8')
                    txt=txt.replace('{classname}',PageName) 
                    txt=txt.replace('{PK_key}',f"{p['PK_QuestionGroup']}")
                    txt=txt.replace('{GroupName}',f"{p['GroupName']}")  
                    if ext=='.aspx': 
                        parsed=parser.parse(question_group=p['PK_QuestionGroup'])
                        code = generator.generate(parsed)
                        txt=txt.replace(f'<!--controls-->',code) 
                    out=out+txt
                path=f"{dest}{p['PNAME']}{ext}"
                print(path)
                if commit:  
                    with open(path, 'w', encoding='UTF-8') as fw:
                        fw.write(txt)
        with open(fr"{ctx.get_dest()}\script.aspx", 'w', encoding='UTF-8') as fw: 
            fw.write(out)
 
dest=rf"C:\\dev\\CyberScope\\CyberScopeBranch\\CSwebdev\\code\\CyberScope\\FismaForms\\2023\\"  
generate_pages(PK_FORM='2023-A-IG', where=' PK_QuestionGroup < 4461 ', commit=True, PageNameFilter='Function',dest=dest)